In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.3/886.3 kB 14.8 MB/s eta 0:00:00


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from datetime import datetime
import csv
import os
from typing import Dict, List, Tuple
from tqdm import tqdm

class EmployeeTracker:
    def __init__(self, input_video_path: str, confidence_threshold: float = 0.5):
        """
        Initialize the employee tracking system for MP4 video processing.

        Args:
            input_video_path: Path to input MP4 video file
            confidence_threshold: Minimum confidence score for detection
        """
        if not os.path.exists(input_video_path):
            raise FileNotFoundError(f"Video file not found: {input_video_path}")

        self.input_video_path = input_video_path
        self.confidence_threshold = confidence_threshold
        self.model = YOLO('yolov8n.pt')  # Load YOLOv8 model
        self.entry_line = None
        self.exit_line = None
        self.tracked_ids = {}

        # Create output directory if it doesn't exist
        self.output_dir = 'output'
        os.makedirs(self.output_dir, exist_ok=True)

        # Initialize file paths
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        self.log_file = os.path.join(self.output_dir, f'employee_movement_{timestamp}.csv')
        self.output_video_path = os.path.join(self.output_dir, f'processed_{timestamp}.mp4')

        self.initialize_log_file()

    def initialize_log_file(self):
        """Create and initialize the CSV log file with headers."""
        with open(self.log_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Timestamp', 'Employee_ID', 'Event', 'Location', 'Frame_Number'])

    def log_movement(self, employee_id: int, event: str, location: str, frame_number: int):
        """Log employee movement events to CSV file."""
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        with open(self.log_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([timestamp, employee_id, event, location, frame_number])

    def setup_monitoring_lines(self, frame_height: int, frame_width: int):
        """
        Set up vertical entry and exit monitoring lines based on frame dimensions.
        Entry line at 1/3 of frame width, exit line at 2/3 of frame width.
        """
        # Vertical lines: (x, y1) to (x, y2)
        self.entry_line = [(frame_width // 3, 0), (frame_width // 3, frame_height)]  # Left line
        self.exit_line = [(2 * frame_width // 3, 0), (2 * frame_width // 3, frame_height)]  # Right line

    def check_line_crossing(self, point: Tuple[int, int], line: List[Tuple[int, int]]) -> bool:
        """
        Check if a point has crossed a vertical line using point-line distance.
        Modified to be more sensitive to vertical line crossings.
        """
        x, y = point
        x1, y1 = line[0]  # Top point of vertical line
        x2, y2 = line[1]  # Bottom point of vertical line

        # For vertical lines, we mainly care about the x-distance
        if min(y1, y2) <= y <= max(y1, y2):  # Check if point is within line's y-range
            return abs(x - x1) < 10  # Using x1 since it's a vertical line (x1 == x2)
        return False

    def process_frame(self, frame: np.ndarray, frame_number: int) -> np.ndarray:
        """
        Process a single frame for person detection and tracking.

        Args:
            frame: Input frame from video
            frame_number: Current frame number for logging

        Returns:
            np.ndarray: Processed frame with visualizations
        """
        # Run YOLOv8 tracking on the frame
        results = self.model.track(frame, persist=True, classes=[0],
                                 conf=self.confidence_threshold)

        if results[0].boxes.id is not None:
            boxes = results[0].boxes.xyxy.cpu().numpy()
            track_ids = results[0].boxes.id.cpu().numpy()

            # Process each detected person
            for box, track_id in zip(boxes, track_ids):
                x1, y1, x2, y2 = box.astype(int)
                track_id = int(track_id)

                center_point = (int((x1 + x2) / 2), int((y1 + y2) / 2))

                # Check for line crossings
                if self.check_line_crossing(center_point, self.entry_line):
                    if track_id not in self.tracked_ids or self.tracked_ids[track_id] != 'entered':
                        self.tracked_ids[track_id] = 'entered'
                        self.log_movement(track_id, 'Entry', f'Position: {center_point}', frame_number)

                elif self.check_line_crossing(center_point, self.exit_line):
                    if track_id not in self.tracked_ids or self.tracked_ids[track_id] != 'exited':
                        self.tracked_ids[track_id] = 'exited'
                        self.log_movement(track_id, 'Exit', f'Position: {center_point}', frame_number)

                # Draw bounding box and ID
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Draw vertical monitoring lines
        cv2.line(frame, self.entry_line[0], self.entry_line[1], (255, 0, 0), 2)  # Blue entry line
        cv2.putText(frame, "Entry", (self.entry_line[0][0] - 30, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        cv2.line(frame, self.exit_line[0], self.exit_line[1], (0, 0, 255), 2)  # Red exit line
        cv2.putText(frame, "Exit", (self.exit_line[0][0] - 30, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Add frame number to the frame
        cv2.putText(frame, f"Frame: {frame_number}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        return frame

    def process_video(self):
        """Process the entire video file and save results."""
        # Open input video
        cap = cv2.VideoCapture(self.input_video_path)
        if not cap.isOpened():
            raise ValueError("Error: Could not open video file")

        # Get video properties
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        # Setup monitoring lines
        self.setup_monitoring_lines(height, width)

        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(self.output_video_path, fourcc, fps, (width, height))

        print(f"\nProcessing video: {self.input_video_path}")
        print(f"Total frames: {total_frames}")
        print(f"FPS: {fps}")
        print(f"Resolution: {width}x{height}")

        try:
            # Process each frame with progress bar
            for frame_number in tqdm(range(total_frames), desc="Processing frames"):
                ret, frame = cap.read()
                if not ret:
                    break

                # Process the frame
                processed_frame = self.process_frame(frame, frame_number)

                # Write the frame to output video
                out.write(processed_frame)

        finally:
            # Clean up
            cap.release()
            out.release()

        print(f"\nProcessing complete!")
        print(f"Output video saved to: {self.output_video_path}")
        print(f"Movement log saved to: {self.log_file}")

        # Generate summary statistics
        self.generate_summary()

    def generate_summary(self):
        """Generate and print summary statistics from the movement log."""
        entries = 0
        exits = 0
        unique_ids = set()

        with open(self.log_file, 'r') as f:
            reader = csv.DictReader(f)
            for row in reader:
                unique_ids.add(row['Employee_ID'])
                if row['Event'] == 'Entry':
                    entries += 1
                elif row['Event'] == 'Exit':
                    exits += 1

        print("\nSummary Statistics:")
        print(f"Total unique IDs tracked: {len(unique_ids)}")
        print(f"Total entries: {entries}")
        print(f"Total exits: {exits}")

if __name__ == "__main__":
    # Example usage
    video_path = "/content/People.mp4"  # Replace with your video path
    tracker = EmployeeTracker(input_video_path=video_path)
    tracker.process_video()


Processing video: /content/People.mp4
Total frames: 320
FPS: 25
Resolution: 1280x720


Processing frames:   0%|          | 0/320 [00:00<?, ?it/s]


0: 384x640 (no detections), 675.4ms
Speed: 7.8ms preprocess, 675.4ms inference, 2.2ms postprocess per image at shape (1, 3, 384, 640)


Processing frames:   0%|          | 1/320 [00:01<05:44,  1.08s/it]


0: 384x640 (no detections), 794.4ms
Speed: 11.2ms preprocess, 794.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)


...


0: 384x640 5 persons, 148.1ms
Speed: 5.2ms preprocess, 148.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing frames:  99%|█████████▉| 316/320 [01:14<00:00,  4.44it/s]


0: 384x640 5 persons, 176.4ms
Speed: 5.0ms preprocess, 176.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


Processing frames:  99%|█████████▉| 317/320 [01:14<00:00,  4.25it/s]


Processing complete!
Output video saved to: output/processed_20241115_223653.mp4
Movement log saved to: output/employee_movement_20241115_223653.csv

Summary Statistics:
Total unique IDs tracked: 11
Total entries: 4
Total exits: 8
